In [1]:
!pip install PySimpleGUI
!pip install -U kaleido
import pandas as pd
import plotly.graph_objects as go
import PySimpleGUI as sg

# Define the GUI layout
layout = [
    [sg.Text('Sankey Diagram')],
    [sg.Text('Upload Data', size=(15, 1)), sg.InputText(key='file_path'), sg.FileBrowse()],
    [sg.Text('Source Column', size=(15, 1)), sg.InputText(key='source_col')],
    [sg.Text('Target Column', size=(15, 1)), sg.InputText(key='target_col')],
    [sg.Text('Value Column', size=(15, 1)), sg.InputText(key='value_col')],
    [sg.Button('Run'), sg.Button('Exit')],
    [sg.Image(key='sankey_plot')]
]

# Create the window
window = sg.Window('Sankey Diagram', layout)

# Event loop
while True:
    event, values = window.read()
    
    if event == 'Exit' or event == sg.WIN_CLOSED:
        break
    
    if event == 'Run':
        # Load the data
        try:
            data = pd.read_csv(values['file_path'])
        except Exception as e:
            sg.popup_error(f'Error loading data: {e}')
            continue
        
        # Create the Sankey diagram
        try:
            fig = go.Figure(data=[go.Sankey(
                node=dict(
                    pad=15,
                    thickness=20,
                    line=dict(color='black', width=0.5),
                    label=data[values['source_col']].drop_duplicates().tolist() + 
                          data[values['target_col']].drop_duplicates().tolist(),
                    color='blue'
                ),
                link=dict(
                    source=data[values['source_col']].map(
                        {node: idx for idx, node in enumerate(data[values['source_col']].unique())}
                    ),
                    target=data[values['target_col']].map(
                        {node: idx + len(data[values['source_col']].unique()) for idx, node in 
                         enumerate(data[values['target_col']].unique())}
                    ),
                    value=data[values['value_col']]
                )
            )])
            fig.update_layout(title='Sankey Diagram', font_size=10)
            sankey_plot = fig.to_image(format='png')
            window['sankey_plot'].update(data=sankey_plot)
        except Exception as e:
            sg.popup_error(f'Error creating Sankey diagram: {e}')
            continue

# Close the window
window.close()
